In [20]:
# Read the 23andMe raw data file and turn it into a dataframe
import pandas as pd
import numpy as np
import sys

ExampleData = "../../Data/23andMe_files/genome_Chad_Wrye_v5_Full_20220921063742.txt"

# Read the text file into a DataFrame
ttAndMe = pd.read_csv(ExampleData, sep='\t', comment='#', header=None, names=['rsid', 'chromosome', 'position', 'genotype'])

# Find all the positions in ttAndMe that are in PRS313 by identifying columns with the same position and chromosome
PRS313 = pd.read_excel('../../Data/PRS313.xlsx')

PRS313['in_23andMe'] = PRS313.apply(lambda x: ttAndMe[(ttAndMe['position'] == x['Positionb']) & (ttAndMe['chromosome'] == x['Chromosome'])].shape[0] > 0, axis=1)

PRS313[(PRS313['in_23andMe'] == True) & (PRS313["Chromosome"] == 1)]

PRS313.to_excel('../../Data/PRS313_with_23andMe.xlsx', index=False)

PRS313

/var/folders/yg/982c10113957_2gb06y92y35sx509h/T/ipykernel_57645/378530466.py:9: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  ttAndMe = pd.read_csv(ExampleData, sep='\t', comment='#', header=None, names=['rsid', 'chromosome', 'position', 'genotype'])


,SNPa,Chromosome,Positionb,Reference Allele,Effect Allele,EAFc,Overall Breast Cancerd,ER-positivee,ER-negativef,hybrid ER-positiveg,hybrid ER-negativeh,in_23andMe,rsid
0,1_100880328_A_T,1,100880328,A,T,0.409678,0.0373,0.0355,0.0160,0.0373,0.0373,False,NaN
1,1_10566215_A_G,1,10566215,A,G,0.329016,-0.0586,-0.0407,-0.1109,-0.0407,-0.1109,True,rs616488
2,1_110198129_CAAA_C,1,110198129,CAAA,C,0.775478,0.0458,0.0545,0.0266,0.0458,0.0458,False,NaN
3,1_114445880_G_A,1,114445880,G,A,0.166363,0.0621,0.0642,0.0579,0.0621,0.0621,False,NaN
4,1_118141492_A_C,1,118141492,A,C,0.265682,0.0452,0.0417,0.0551,0.0452,0.0452,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
308,22_39343916_T_A,22,39343916,T,A,0.254083,0.0407,0.0326,0.0330,0.0407,0.0407,False,NaN
309,22_40904707_CT_C,22,40904707,CT,C,0.109871,0.1148,0.1160,0.1203,0.1148,0.1148,False,NaN
310,22_43433100_C_T,22,43433100,C,T,0.114445,-0.0600,-0.0585,-0.0515,-0.0600,-0.0600,False,NaN
311,22_45319953_G_A,22,45319953,G,A,0.416619,-0.0134,-0.0060,-0.0611,-0.0060,-0.0611,False,NaN


In [33]:
import os
import pandas as pd

# Set the path to the folder containing the 23andMe files
folder_path = "../../Data/23andMe_files"

# Read the PRS313 file
PRS313 = pd.read_excel('../../Data/PRS313.xlsx')

# Initialize a DataFrame to store the results
result_df = pd.DataFrame(columns=PRS313['SNPa'].tolist() + ['total_snps_present', 'total_positions'])

# Initialize a DataFrame to store the unique positions from all 23andMe files
unique_positions_df = pd.DataFrame(columns=['rsid', 'chromosome', 'position'])

# Initialize a list to store the positions from each 23andMe file
positions_list = []

# Iterate over each file in the folder
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    try:
        # Read the 23andMe raw data file into a DataFrame
        ttAndMe = pd.read_csv(file_path, sep='\t', comment='#', header=None, names=['rsid', 'chromosome', 'position', 'genotype'])

        # If the number of positions is greater than 700000 or less than 600000, delete the file and move on
        if ttAndMe.shape[0] > 700000 or ttAndMe.shape[0] < 600000:
            os.remove(file_path)
            continue

        # Initialize a dictionary to store the presence of each SNP
        snp_presence = {}

        # Check the presence of each SNP in the 23andMe file
        for _, row in PRS313.iterrows():
            snp_location = row['SNPa']
            chromosome = row['Chromosome']
            position = row['Positionb']
            snp_presence[snp_location] = ttAndMe[(ttAndMe['position'] == position) & (ttAndMe['chromosome'] == chromosome)].shape[0] > 0

        # Count the total number of SNPs present
        total_snps_present = sum(snp_presence.values())

        # Count the total number of positions in the 23andMe file
        total_positions = ttAndMe.shape[0]

        # Add the results to the DataFrame
        result_df.loc[filename] = [snp_presence[snp_location] for snp_location in PRS313['SNPa'].tolist()] + [total_snps_present, total_positions]

        # Append the positions from the current 23andMe file to the positions_list
        positions_list.append(ttAndMe[['rsid', 'chromosome', 'position']])

    except Exception as e:
        print(f"Error processing file {filename}: {e}")
        continue

# Concatenate all the positions from each 23andMe file
all_positions_df = pd.concat(positions_list, ignore_index=True)

# Get the unique positions from all 23andMe files
unique_positions_df = all_positions_df.drop_duplicates(subset=['chromosome', 'position', 'rsid'])

# Find the intersection of positions present in all 23andMe files
intersection_positions_df = all_positions_df.groupby(['chromosome', 'position']).filter(lambda x: len(x) == len(positions_list))

# Save the results to Excel files
result_df.to_excel('../../Data/PRS313_23andMe_results.xlsx', index_label='filename')
unique_positions_df.to_excel('../../Data/23andMe_unique_positions.xlsx', index=False)
intersection_positions_df.to_excel('../../Data/23andMe_intersection_positions.xlsx', index=False)

/var/folders/yg/982c10113957_2gb06y92y35sx509h/T/ipykernel_57645/679208585.py:18: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  ttAndMe = pd.read_csv(file_path, sep='\t', comment='#', header=None, names=['rsid', 'chromosome', 'position', 'genotype'])
/var/folders/yg/982c10113957_2gb06y92y35sx509h/T/ipykernel_57645/679208585.py:18: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  ttAndMe = pd.read_csv(file_path, sep='\t', comment='#', header=None, names=['rsid', 'chromosome', 'position', 'genotype'])
/var/folders/yg/982c10113957_2gb06y92y35sx509h/T/ipykernel_57645/679208585.py:18: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  ttAndMe = pd.read_csv(file_path, sep='\t', comment='#', header=None, names=['rsid', 'chromosome', 'position', 'genotype'])
/var/folders/yg/982c10113957_2gb06y92y35sx509h/T/ipykernel_57645/679208585.py:1

Error processing file 12330.23andme.10166.txt: 'utf-8' codec can't decode byte 0xff in position 15: invalid start byte


/var/folders/yg/982c10113957_2gb06y92y35sx509h/T/ipykernel_57645/679208585.py:18: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  ttAndMe = pd.read_csv(file_path, sep='\t', comment='#', header=None, names=['rsid', 'chromosome', 'position', 'genotype'])
/var/folders/yg/982c10113957_2gb06y92y35sx509h/T/ipykernel_57645/679208585.py:18: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  ttAndMe = pd.read_csv(file_path, sep='\t', comment='#', header=None, names=['rsid', 'chromosome', 'position', 'genotype'])
/var/folders/yg/982c10113957_2gb06y92y35sx509h/T/ipykernel_57645/679208585.py:18: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  ttAndMe = pd.read_csv(file_path, sep='\t', comment='#', header=None, names=['rsid', 'chromosome', 'position', 'genotype'])


Error processing file 12335.23andme.10172.txt: 'utf-8' codec can't decode byte 0x8b in position 1: invalid start byte


/var/folders/yg/982c10113957_2gb06y92y35sx509h/T/ipykernel_57645/679208585.py:18: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  ttAndMe = pd.read_csv(file_path, sep='\t', comment='#', header=None, names=['rsid', 'chromosome', 'position', 'genotype'])
/var/folders/yg/982c10113957_2gb06y92y35sx509h/T/ipykernel_57645/679208585.py:18: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  ttAndMe = pd.read_csv(file_path, sep='\t', comment='#', header=None, names=['rsid', 'chromosome', 'position', 'genotype'])
/var/folders/yg/982c10113957_2gb06y92y35sx509h/T/ipykernel_57645/679208585.py:18: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  ttAndMe = pd.read_csv(file_path, sep='\t', comment='#', header=None, names=['rsid', 'chromosome', 'position', 'genotype'])
/var/folders/yg/982c10113957_2gb06y92y35sx509h/T/ipykernel_57645/679208585.py:1

Error processing file 12373.23andme.10203.txt: 'utf-8' codec can't decode byte 0x8b in position 1: invalid start byte


/var/folders/yg/982c10113957_2gb06y92y35sx509h/T/ipykernel_57645/679208585.py:18: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  ttAndMe = pd.read_csv(file_path, sep='\t', comment='#', header=None, names=['rsid', 'chromosome', 'position', 'genotype'])
/var/folders/yg/982c10113957_2gb06y92y35sx509h/T/ipykernel_57645/679208585.py:18: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  ttAndMe = pd.read_csv(file_path, sep='\t', comment='#', header=None, names=['rsid', 'chromosome', 'position', 'genotype'])
/var/folders/yg/982c10113957_2gb06y92y35sx509h/T/ipykernel_57645/679208585.py:18: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  ttAndMe = pd.read_csv(file_path, sep='\t', comment='#', header=None, names=['rsid', 'chromosome', 'position', 'genotype'])


KeyboardInterrupt: 

In [29]:
import pandas as pd

# Create frequency table with bins for total_snps_present
bins = [0, 59, 71, 150, 200, 250, 300, 350]
labels = ['0-59', '60-70', '71-150', '151-200', '201-250', '251-300', '301-350']
result_df['total_snps_present_bins'] = pd.cut(result_df['total_snps_present'], bins=bins, labels=labels, right=False)
total_snps_present_freq = result_df['total_snps_present_bins'].value_counts().sort_index()

print("Frequency Table for Total SNPs Present:")
print(total_snps_present_freq)
print()

# Create frequency table with bins for total_positions
bins = [0, 550000, 700000, 1500000, 2000000, 2500000, 10000000]
labels = ['0-550k', '550k-700k', '700k-1.5M', '1.5M-2M', '2M-2.5M', '2.5M-10M']
result_df['total_positions_bins'] = pd.cut(result_df['total_positions'], bins=bins, labels=labels, right=False)
total_positions_freq = result_df['total_positions_bins'].value_counts().sort_index()

print("Frequency Table for Total Positions:")
print(total_positions_freq)

Frequency Table for Total SNPs Present:
total_snps_present_bins
0-59        9
60-70      28
71-150      3
151-200     0
201-250     0
251-300     1
301-350     0
Name: count, dtype: int64

Frequency Table for Total Positions:
total_positions_bins
0-550k        1
550k-700k    34
700k-1.5M     4
1.5M-2M       0
2M-2.5M       1
2.5M-10M      1
Name: count, dtype: int64


In [31]:
# Get the percentage of present snps between 60-70
total_snps_present_freq['60-70'] / result_df.shape[0] * 100

68.29268292682927

# Convert all the 23andMe files to text files

In [32]:
# Set the path to the folder containing the 23andMe files
folder_path = "../../Data/23andMe_files"

# Convert all the 23andMe files to .txt format if they are not already in that format

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    
    # Check if the file is already in .txt format
    if not filename.endswith('.txt'):
        new_file_path = os.path.join(folder_path, filename + '.txt')
        os.rename(file_path, new_file_path)
        print(f"Renamed {filename} to {filename.split('.')[0] + '.txt'}")
        

Renamed 12367.23andme.10198 to 12367.txt
Renamed 12312.23andme.10143 to 12312.txt
Renamed 12343.23andme.10178 to 12343.txt
Renamed 12232.23andme.10180 to 12232.txt
Renamed 12325.23andme.10160 to 12325.txt
Renamed 12332.23andme.10169 to 12332.txt
Renamed 12344.23andme.10179 to 12344.txt
Renamed 12334.23andme.10171 to 12334.txt
Renamed 3033.23andme.10150 to 3033.txt
Renamed 12335.23andme.10172 to 12335.txt
Renamed 12356.23andme.10187 to 12356.txt
Renamed 12370.23andme.10200 to 12370.txt
Renamed 12320.23andme.10152 to 12320.txt
Renamed 12368.23andme.10199 to 12368.txt
Renamed 12348.23andme.10183 to 12348.txt
Renamed 12337.23andme.10181 to 12337.txt
Renamed 12363.23andme.10193 to 12363.txt
Renamed 4015.23andme.10141 to 4015.txt
Renamed 12371.23andme.10201 to 12371.txt
Renamed 12309.23andme.10142 to 12309.txt
Renamed 12308.23andme.10140 to 12308.txt
Renamed 12365.23andme.10196 to 12365.txt
Renamed 12330.23andme.10166 to 12330.txt
Renamed 12342.23andme.10176 to 12342.txt
Renamed 12351.23andm